# HPC/QC Architectures: What is Quantum Programming?

Writing quantum code requires different ideas and skills. It is important to understand that it is still very low level compared to classical programming.

When was the last time you wrote your full-adder to implement `a + b` (per bit!)?

**Classic Full Adder Circuit** 

<img src="images/full-adder-circuit.png" width="30%">


Yet, this is the level which we are working in quantum programming today:

**Quantum Full Adder Circuit** 

<img src="images/full-adder-qcircuit.png" width="30%">

## Primitives in Programming

> In computing, language primitives are the simplest elements available in a programming language. A primitive is the smallest 'unit of processing' available to a programmer of a given machine, or can be an atomic element of an expression in a language. [Source: Wikipedia](https://en.wikipedia.org/wiki/Language_primitive)

Primitives come at different levels or flavors, for example:
* NOT, AND, NAND, OR, NOR, XOR etc are primitive logic gates implemented in electronic circuits
* Microcode instructions are primitives of the hardware itself, directly interfacing the electronic circuits
* Assembler instructions are machine-level primitives in classic programming available to programmers
* Statements and constructs such as `for`, `while`, `+`, `-` are primitives of high-level programming languages
* Methods such as `dgemm` (matrix-matrix product), `dgemv` (matrix-vector product) are primitives of high-level libraries



## Primitives in Quantum Programming 
Quantum Programming still requires us to work with lowest-level primitives such as elementary quantum gates.

But development of higher level abstractions and algorithms is evolving fast and more and more re-usable 'circuit blocks' are made available through libraries.

Example: [Qiskit Algorithms](https://docs.quantum.ibm.com/api/qiskit/0.33/algorithms) which contains re-usable circuits for tasks such as
* Amplitude amplification
* Linear Solvers
* Eigensolvers
* Phase Estimators

Example: [Qiskit Circuit Library](https://docs.quantum.ibm.com/api/qiskit/circuit_library) which contains lower-level primitives for many tasks such as
* basic gates (e.g., H, X, T, CX)
* composed gates (e.g., CCX, C4X)
* generalized and parametrized gates
* Quantum Fourier Transform
* Arithmetic operations (adders, multipliers, ...)

## Steps of a Quantum Algorithm
Quantum algorithms make use of the superposition and (relative) phases of qubits to solve problems. The answer is usually encoded in the superposition state and not easily accessibly for read operations.




<img src="images/quantum_programming.svg" width="60%">

### Create initial state with superposition
Qubits can be used exactly as classical bits (representing registers of bytes, logic operations etc).

**Problem:** This would not give us any quantum advantage, only a very slow, very ineffective and very expensive computer.

&rightarrow; Our data must be *encoded* in the qubit registers in a way that entanglement and superposition can be used

#### What can be *encoded*?
* numbers
* vectors
* matrices
  
Or on (slightly) higher-level:
* Hamiltonians
* Optimization Problems (3-SAT etc.)
* ... and much more

#### How can it be encoded?
In contrast to classic computation it is also possible to encode information in the (complex) amplitudes and (relative) phase differences of qubits

**&rightarrow; More later!**

### Compute in Superposition
Quantum computation is different from classic computation. Many of the known rules don't apply or won't give any computational advantage.

#### Constraints of Quantum Computation
**No-Copy**
* Qubits cannot be copied
* Classic algorithms rely heavily on copying and assigning values to different bits

**Reversibility**
* By definition, all operations performed on a quantum computer must be reversible
* Often not the case for classic logic, e.g., `c = a+b` irreversibly overwrites state of `c` and cannot be undone

### Phase Manipulation
**Important:** The phase of the qubits will have no effect on the measurement in the standard computational basis!

* Only amplitudes have effect on measured value 
* Phase manipulation plays a crucial role in many quantum algorithms (e.g., QFT, Phase Estimation)

#### Example Phase & Measurement
The following example measures a single qubit superposition state with a phase difference

In [ ]:
from qiskit import Aer, QuantumCircuit
from qiskit.tools.visualization import plot_histogram, plot_bloch_multivector
from qiskit import quantum_info
import numpy as np

simulator = Aer.get_backend('aer_simulator')

# Create a simple circuit with qubit in superposition
phase = QuantumCircuit(1)
phase.h(0)

# Get the state vector in superposition
st1 = quantum_info.Statevector(phase)

# Apply a pi/3 rotation around z and store state
phase.rz(np.pi/3, 0)
st2 = quantum_info.Statevector(phase)

phase.measure_all()
phase.draw(output="mpl", style="iqp")

In [ ]:
result = simulator.run(phase, shots=100000).result()
plot_histogram(result.get_counts())

Looking at the Bloch representation of both single-qubit states it becomes clear that the phase has no effect on the probability of measuring 0 or 1

In [ ]:
plot_bloch_multivector(st1)

In [ ]:
plot_bloch_multivector(st2)

### Interesting Phase Effects: Phase Kickback
Phase kickback is an effect that plays a role in more complex quantum algorithms (Quantum Phase Estimation or Quantum Machine Learning). It allows the phase of a control qubit to be changed by an operation performed on some other target qubit.

#### Application: Bi-directional CNOT gates
On some (older) hardware it was not possible to swap control and target qubit for a CNOT gate. Only one direction (e.g., Control(0), Target(1)) was possible.

Using phase kickback one can create a circuit to implement the other direction, Control(1) and Target(0) using the same CNOT gate direction.

#### Normal CNOT operation between control qubit q0 (state 1) and target q1 (state 0) to give 11

In [ ]:
circ = QuantumCircuit(2)
circ.x(0)
circ.barrier()
circ.cx(0,1)
circ.measure_all()
circ.save_statevector()
circ.draw(output="mpl", style="iqp")


In [ ]:
simulator.run(circ).result().get_counts()

#### Phase-Kickback operation between control qubit q0 (state 0) and target q1 (state 1) to give 11

In [ ]:
circ = QuantumCircuit(2)
circ.x(1)
circ.barrier()
circ.h(0)
circ.h(1)
circ.cx(0,1)
circ.h(0)
circ.h(1)
circ.measure_all()
circ.save_statevector()
circ.draw(output="mpl", style="iqp")


In [ ]:
simulator.run(circ).result().get_counts()

### Readout Results
Readout is a non-reversible operation that will collpase the quantum state and return our result in a classic bit string.

## Summary: Means of Computation
In  Quantum Programming we can differentiate gate operations based on the effects on the qubit: Amplitude-manipulation and Phase-manipulation.

### Amplitude manipulation
* Has a direct (measureable) effect on the outcome of our computation (probability to measure 0 or 1)

### Phase manipulation
* Has no measureable effect
* Information encoded in relative phases is not accessible to us
* Algorithms to move information from "phase space" to "amplitude space" exist (Amplitude Amplification, QFT)